In [1]:
# 불필요한 Warning 제거
import warnings
warnings.filterwarnings('ignore')

In [5]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

2024-12-06 23:10:02.243954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733494202.386181     905 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733494202.428850     905 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-06 23:10:02.783787: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Flan-T5 model 를 이용한 생성
```
model (T5ForConditionalGeneration): Loaded model
tokenizer (T5Tokenizer): Model tokenizer
prompt (str): Input prompt
max_length (int): Maximum length of generated text
```

In [6]:
def generate_text(model, tokenizer, prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt",
                       max_length=512, truncation=True).to(model.device)

    # 평가 모드 이용시 기존의 역전파 이용시 꺼져있는 노드를 다시 켜 예측시에 모든 노드들이 적용되게 한다.
    model.eval() 
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,
        # temperature=0.7,
        temperature = 0.3,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [8]:
def load_flan_t5_model():
    """
    Download and load the Flan-T5-XL model and tokenizer

    Returns:
    tuple: (model, tokenizer)
    """
    # Load the pre-trained model and tokenizer
    model_name = "google/flan-t5-xl"
    # model_name = "google/flan-t5-large"

    # Load tokenizer
    tokenizer = T5Tokenizer.from_pretrained(model_name)

    # Load model (using float16 for memory efficiency)
    model = T5ForConditionalGeneration.from_pretrained(
        model_name,
        torch_dtype=torch.float16,  # Reduces memory usage
        device_map="auto"  # Automatically distributes model across available devices
    )


    print("model == ", model)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model.to(device);

    return model, tokenizer

In [9]:
# Load the model and tokenizer
model, tokenizer = load_flan_t5_model()

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

model ==  T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=Fals

In [32]:
prompts = ["How about getting a Master’s degree? I want to learn more about AI, so I want to dive deeper into the field, even though my major is automotive mechanical engineering. What do you think?"]

for prompt in prompts:
    print(f"Prompt: {prompt}")
    response = generate_text(model, tokenizer, prompt)
    print(f"Response: {response}\n")

Prompt: How about getting a Master’s degree? I want to learn more about AI, so I want to dive deeper into the field, even though my major is automotive mechanical engineering. What do you think?
Response: I should get a Master’s degree in AI.

